In [1]:
#including all the necessary libraries
import torch
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import argparse
import os
import copy
import torch
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
import cv2

/storage/home/hpaceice1/vkondepudi3/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LABELS_Severity = {35: 0, 43: 0, 47: 1, 53: 1, 61: 2, 65: 2, 71: 2, 85: 2}
mean = (.1706)
std = (.2112)
normalize = transforms.Normalize(mean=mean, std=std)

transform = transforms.Compose([transforms.Resize(size=(224,224)),transforms.ToTensor(),normalize,])

In [3]:
class OCTDataset(Dataset):
    def __init__(self, args, subset='train', transform=None,):
        if subset == 'train':
            self.annot = pd.read_csv(args.annot_train_prime)
        elif subset == 'test':
            self.annot = pd.read_csv(args.annot_test_prime)
            
        self.annot['Severity_Label'] = [LABELS_Severity[drss] for drss in copy.deepcopy(self.annot['DRSS'].values)] 
        # print(self.annot)
        self.root = os.path.expanduser(args.data_root)
        self.transform = transform
        # self.subset = subset
        self.nb_classes=len(np.unique(list(LABELS_Severity.values())))
        self.path_list = self.annot['File_Path'].values
        self._labels = self.annot['Severity_Label'].values
        assert len(self.path_list) == len(self._labels)
        # idx_each_class = [[] for i in range(self.nb_classes)]

    def __getitem__(self, index):
        img, target = Image.open(self.root+self.path_list[index]).convert("L"), self._labels[index]

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self._labels)         

In [4]:
class NotebookArgs:
    def __init__(self, annot_train_prime = 'df_prime_train.csv', annot_test_prime = 'df_prime_test.csv', data_root = '/storage/home/hpaceice1/shared-classes/materials/ece8803fml/'):
        self.annot_train_prime = annot_train_prime
        self.annot_test_prime = annot_test_prime
        self.data_root = data_root
args = NotebookArgs()

In [5]:
#loading the dataset
trainset = OCTDataset(args, 'train', transform=transform)
testset = OCTDataset(args, 'test', transform=transform)

# Define hyperparameters
batch_size = 32

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)

In [6]:
def get_X_y_from_loader(loader):
    X, y = [], []
    i=0
    for sample in tqdm(loader, total=len(loader)):
        i = i+1
        images, labels = sample[0], sample[1]
        X.extend([a.numpy()[0] for a in images])
        y.extend([a.numpy().flatten() for a in labels])
        #break
        #if(i == 1000):
           #break
    return X,y
X_train, y_train = get_X_y_from_loader(train_loader)
print(len(X_train))

100%|██████████| 758/758 [32:53<00:00,  2.60s/it]

24252


In [7]:
#applying sift feature extraction
X_train_sift = []
i=0
for i in tqdm(range(len(X_train))):   
    #X_hog = X_train[i].reshape((224,224))
    img_uint8 = cv2.convertScaleAbs(X_train[i])
    sift = cv2.SIFT_create(nfeatures=50, contrastThreshold=0.001, edgeThreshold=5)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(img_uint8, None)
    descriptors=descriptors.flatten()
    descriptors= descriptors[:6400] 
    X_train_sift.append(descriptors)
    i =i+1
    if i==10000:
        break
print(X_train_sift[0].shape)
print(len(X_train_sift))

 41%|████      | 9999/24252 [01:48<02:34, 92.37it/s]

(6400,)
10000


In [8]:
print(len(y_train))
X_subset = X_train_sift
y_subset = y_train[0:10000]
print(len(y_subset))
#for arr in (X_subset):
    #print(arr.shape)


24252
10000


In [9]:
X_test, y_test = get_X_y_from_loader(test_loader)

100%|██████████| 250/250 [10:22<00:00,  2.49s/it]


In [10]:
X_test_sift = []
for i in tqdm(range(len(X_test))):
    #X_hog = X_train[i].reshape((224,224))
    img_uint8 = cv2.convertScaleAbs(X_test[i])
    sift = cv2.SIFT_create(nfeatures=50, contrastThreshold=0.001, edgeThreshold=5)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(img_uint8, None)
    descriptors = descriptors.flatten()
    descriptors = descriptors[:6400]
    X_test_sift.append(descriptors)
print(X_test_sift[0].shape)
print(len(X_test_sift))

100%|██████████| 7987/7987 [02:20<00:00, 56.83it/s]

(6400,)
7987


In [15]:
y_subset = [i[0] for i in y_subset]
y_test = [i[0] for i in y_test]

In [16]:
#performing feed forward
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Define the number of input features
input_dim = len(X_subset[0])

# Define the number of output classes
num_classes = 3

# Convert the training and test sets to PyTorch tensors
X_train_tensor = torch.Tensor(X_subset)
y_train_tensor = torch.LongTensor(y_subset)
X_test_tensor = torch.Tensor(X_test_sift)
y_test_tensor = torch.LongTensor(y_test)

# Define the model architecture
model = nn.Sequential(
    nn.Linear(input_dim, 1024),
    nn.ReLU(),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, num_classes),
    nn.Softmax(dim=1)
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Define the training data loader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Epoch:', epoch+1, 'Loss:', loss.item())

# Evaluate the model on the test data
with torch.no_grad():
    output = model(X_test_tensor)
    _, predicted = torch.max(output.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test)
    print('Test accuracy:', accuracy)


Epoch: 1 Loss: 0.8810282945632935
Epoch: 2 Loss: 1.0281893014907837
Epoch: 3 Loss: 1.0130897760391235
Epoch: 4 Loss: 0.9264447689056396
Epoch: 5 Loss: 0.9889447689056396
Epoch: 6 Loss: 1.0514447689056396
Epoch: 7 Loss: 1.1139448881149292
Epoch: 8 Loss: 1.0514447689056396
Epoch: 9 Loss: 1.1764447689056396
Epoch: 10 Loss: 1.0514447689056396
Test accuracy: 0.49079754601226994


In [14]:
print(target.shape)
print(output.shape)
print(target[0])
print(output[0])

torch.Size([32, 1])
torch.Size([32, 3])
tensor([0])
tensor([0.4602, 0.2737, 0.2661], grad_fn=<SelectBackward0>)


In [19]:
#printing all the metrics
from sklearn.metrics import precision_score, recall_score, balanced_accuracy_score

with torch.no_grad():
    output = model(X_test_tensor)
    _, predicted = torch.max(output.data, 1)
    predicted = predicted.cpu()
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f'Test accuracy on {len(predicted)} points is {accuracy}')
    f1 = f1_score(y_test_tensor.cpu().numpy(), predicted.numpy(), average='weighted')
    precision = precision_score(y_test_tensor.cpu().numpy(), predicted.numpy(), average='weighted')
    recall = recall_score(y_test_tensor.cpu().numpy(), predicted.numpy(), average='weighted')
    balanced_accuracy = balanced_accuracy_score(y_test_tensor.cpu().numpy(), predicted.numpy())
    print(f'f1 score on {len(predicted)} points is {f1}')
    print(f'Test precision on {len(predicted)} points is {precision}')
    print(f'Test recall on {len(predicted)} points is {recall}')
    print(f'Test balanced accuracy on {len(predicted)} points is {balanced_accuracy}')

Test accuracy on 7987 points is 0.49079754601226994
f1 score on 7987 points is 0.3231588780327703
Test precision on 7987 points is 0.2408822311716662
Test recall on 7987 points is 0.49079754601226994
Test balanced accuracy on 7987 points is 0.3333333333333333


/storage/home/hpaceice1/vkondepudi3/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
